In [18]:
##set up
import numpy as np
import pandas as pd
import string
import email
import os ##use this to traverse folders
from email.parser import Parser
import re
import lxml.html
import collections

In [19]:
#extract labels from trec06p/full/ file lists
with open('trec06p/full/index', 'r') as ifile:
    raw_labels = ifile.readlines()

#using file list to give labels and e-mail numbers

categories = [i.split( )[0] for i in raw_labels]
email_nums = [i.split(".")[-1][:-1] for i in raw_labels]
labels = pd.Series(categories, index=email_nums)

In [20]:
# extract the body of email
def walkMsg(msg):
    """
    Input: string of email
    Output: body of email
    """
    for part in msg.walk():
        if part.get_content_type() == "multipart/alternative":
            continue
        yield part.get_payload(decode=1)

In [21]:
##encode
def robust_decode(bs):
    '''Takes a byte string as param and convert it into a unicode one.
First tries UTF8, and fallback to Latin1 if it fails'''
    cr = None
    try:
        #cr = bs.decode('utf8')
        cr = bs.decode('utf-8-sig')
    except UnicodeDecodeError:
        cr = bs.decode('ISO-8859-1')
        cr = cr.encode("ascii", "ignore")
        #cr = cr.decode('utf-8-sig')
    return cr
#.replace("\n"," ").replace("\t"," ").replace("r"," ")

In [28]:
def info_of_email_2():
    """
    Input: path, list of files name, classification(spam or ham)
    Ourput: a list with info of email
    """
    
    parser = Parser()
    info = []
    for nums in range(len(categories)):
        filepath = 'trec06p%s' % email_nums[nums]
                
        with open(filepath, 'r') as ifile:
            email = parser.parse(ifile)
            
            Date = email.get("Date")

            Content = " ".join([robust_decode(i) for i in walkMsg(email) if i != None])
            Content = ''.join(Content.splitlines())
            Content = Content.replace("\t","")
            
            if 'html' in Content:
                try:
                    Content = lxml.html.fromstring(Content).text_content()
                except: 
                    Content = Content
            try:
                Date = str.split(Date)
                Day = Date[0].replace(',','')
                Hour = Date[4].split(':')[0]
                info.append([Date[3], Date[2], Date[1],Day, Hour,
                             email.get("From"),email.get('To'),
                             email.get('cc'),email.get('bcc'), 
                             str(email.get("Content-Type")).split(";")[0],
                             email.get('subject'),
                             Content,
                             categories[nums]])
            except:
                info.append([[], [], [], [], [],
                             email.get("From"), email.get('To'),
                             email.get('cc'), email.get('bcc'), 
                             str(email.get("Content-Type")).split(";")[0],
                             email.get('subject'),
                             Content,
                             categories[nums]])
    return info

In [29]:
df = pd.DataFrame(info_of_email_2(),
                 columns=['Year','Month','Day','Weekday',"Hour",
                                'From','To','cc','bcc','Content_type','Subjects','Content','Classification'])

In [46]:
df.shape

(37822, 13)

In [30]:
df.head()

,Year,Month,Day,Weekday,Hour,From,To,cc,bcc,Content_type,Subjects,Content,Classification
0,1992,Jul,28,Tue,03,jhpb@sarto.budd-lake.nj.us (Joseph H. Buehler),None,None,None,text,new Catholic mailing list now up and running,The mailing list I queried about a few weeks a...,ham
1,1993,Apr,3,Sat,10,"""Stella Lowry"" <rookcuduq@yahoo.com>","""Brian"" <bernice@groucho.cs.psu.edu>",None,None,multipart/alternative,re[12]:,...,spam
2,1993,Apr,6,Tue,20,"""Walter"" <trwmpca@downtowncumberland.com>",<arline@groucho.cs.psu.edu>,None,None,text/plain,Take a moment to explore this.,Academic Qualifications available from prestig...,spam
3,1993,Apr,9,Fri,14,Scott Schwartz <schwartz@groucho.cs.psu.edu>,9fans <plan9-fans@cs.psu.edu>,None,None,None,Greetings,Greetings all. This is to verify your subscri...,ham
4,1993,Apr,9,Fri,21,"""Mr Jailyn Koepke"" <kiflsbizc@attheworld.com>",melvin@groucho.cs.psu.edu,"jenna@groucho.cs.psu.edu, keri@groucho.cs.psu....",None,multipart/alternative,LOANS @ 3.17% (27 term),try chauncey may conferred the luscious not co...,spam


In [31]:
##need to figure out the way to deal with wrong year/mo/day....##
collections.Counter(df.iloc[[len(i)==4 for i in df['Year']]]['Year'])

Counter({'0102': 4,
         '1889': 6,
         '1890': 2,
         '1893': 18,
         '1894': 41,
         '1895': 25,
         '1896': 16,
         '1897': 6,
         '1898': 12,
         '1899': 4,
         '1900': 12,
         '1901': 29,
         '1902': 3,
         '1903': 2,
         '1904': 3,
         '1906': 2,
         '1926': 4,
         '1927': 3,
         '1928': 1,
         '1929': 1,
         '1930': 15,
         '1931': 37,
         '1932': 34,
         '1933': 3,
         '1934': 6,
         '1935': 2,
         '1936': 5,
         '1937': 19,
         '1938': 17,
         '1939': 3,
         '1956': 1,
         '1980': 2,
         '1987': 2,
         '1991': 2,
         '1992': 7,
         '1993': 284,
         '1994': 396,
         '1995': 252,
         '1996': 184,
         '1997': 353,
         '1998': 6691,
         '1999': 7357,
         '2000': 4409,
         '2001': 1088,
         '2002': 1700,
         '2003': 1559,
         '2004': 1802,
         '2005': 

In [60]:
df[df.Year == "1999"]

,Year,Month,Day,Weekday,Hour,From,To,cc,bcc,Content_type,Subjects,Content,Classification
3950,1999,Mar,19,Fri,12,=?iso-2022-jp?B?GyRCQWpCdCRKJFsbKEI=?= <aizawa...,undisclosed-recipients:;,None,None,text/plain,=?iso-2022-jp?B?GyRCJCQkXiRAJEshIhsoQg==?=,"$B$$$^$@$K%b%6%$%/$""$j$N%l%s%?%k%S%G%*$GK~B-$...",spam
4209,1999,Mar,31,Wed,02,=?iso-2022-jp?B?GyRCQWpCdCRKJFsbKEI=?= <aizawa...,undisclosed-recipients:;,None,None,text/plain,=?iso-2022-jp?B?GyRCJCQkXiRAJEshIhsoQg==?=,"$B$$$^$@$K%b%6%$%/$""$j$N%l%s%?%k%S%G%*$GK~B-$...",spam
4482,1999,Apr,13,Tue,04,=?iso-2022-jp?B?GyRCQWpCdCRKJFsbKEI=?= <aizawa...,undisclosed-recipients:;,None,None,text/plain,=?iso-2022-jp?B?GyRCJCQkXiRAJEshIhsoQg==?=,"$B$$$^$@$K%b%6%$%/$""$j$N%l%s%?%k%S%G%*$GK~B-$...",spam
4646,1999,Apr,21,Wed,12,=?iso-2022-jp?B?GyRCQWpCdCRKJFsbKEI=?= <aizawa...,undisclosed-recipients:;,None,None,text/plain,=?iso-2022-jp?B?GyRCJCQkXiRAJEshIhsoQg==?=,"$B$$$^$@$K%b%6%$%/$""$j$N%l%s%?%k%S%G%*$GK~B-$...",spam
9290,1999,Jan,1,Fri,11,info@oh-oku.com,handyboard@media.mit.edu,None,None,text/plain,"$BK-K~$J$*$C$Q$$""v$r;H$C$F5$;}$A$h$/!D(B","$B""""!?""#!?""""!?(B$B!?""""!?""#!?!!!!>.(B$B""#!?""""!?...",spam
9293,1999,Jan,1,Fri,11,"""Timothy Stephenson"" <mbhxhfzmq@eos.ocn.ne.jp>","handyboard@media.mit.edu, earline@media.mit.ed...",None,None,text/plain,Boosts Energy,"Summer is coming, did you look in the mirror l...",spam
9294,1999,Jan,1,Fri,07,"""Audra Russo"" <care@0toall-research-paper-topi...",chelsea@aleve.media.mit.edu,None,None,text/plain,All products for your health!,http://kiwfgf.winslogan.com/?99578557Suffering...,spam
9297,1999,Jan,1,Fri,21,"""SHETTI.NITIN.MANGESH"" <shettin@giasbmc.vsnl.n...",handyboard@media.mit.edu,None,None,TEXT/PLAIN,DEBUGGING SOFTWARE,"Dear friends, With respect to page 31(page n...",ham
9298,1999,Jan,01,Fri,10,"""K.L.McKinnon"" <""klmckin@nospam""@magmacom.com>",Michael Toh <leahcim@pacific.net.sg>,handyboard@media.mit.edu,None,text/plain,Re: sensors input,You don't really show your connection to your ...,ham
9299,1999,Jan,1,Fri,12,"""Ron Chase"" <ronchase@pobox.com>","""Handyboard"" <handyboard@media.mit.edu>",None,None,text/plain,RE: Announcing the Handy Board Expansion Board,How can I get ahold of a Lego shaft encoder?> ...,ham


In [32]:
df.iloc[[len(i)!=4 for i in df['Year']]]

,Year,Month,Day,Weekday,Hour,From,To,cc,bcc,Content_type,Subjects,Content,Classification
29,[],[],[],[],[],=?iso-2022-jp?B?d3JxZA==?=<qdrq@yahoo.co.hk>,<winnie@groucho.cs.psu.edu>,None,None,text/plain,=?iso-2022-jp?B?GyRCOSUkKyRsJGs/TSFaJVslQyVIJW...,$B=w$N;R$+$iCK$N;R$X%*%9%9%a$9$k%J%$%9$JJ8=qH...,spam
55,[],[],[],[],[],=?iso-2022-jp?B?d3JxZA==?=<qdrq@yahoo.co.hk>,<winnie@groucho.cs.psu.edu>,None,None,text/plain,=?iso-2022-jp?B?GyRCOSUkKyRsJGs/TSFaJVslQyVIJW...,$B=w$N;R$+$iCK$N;R$X%*%9%9%a$9$k%J%$%9$JJ8=qH...,spam
67,[],[],[],[],[],"""houndog42"" <tgfyneucbjy@hotmail.com>",rosalie@groucho.cse.psu.edu,None,None,text/html,After Penis Enlarge Patch you cock won�t resem...,99 percent of <ach></ach>all men have a weaker...,spam
91,[],[],[],[],[],"""houndog42"" <tgfyneucbjy@hotmail.com>",rosalie@groucho.cse.psu.edu,None,None,text/html,After Penis Enlarge Patch you cock won�t resem...,99 percent of <ach></ach>all men have a weaker...,spam
105,[],[],[],[],[],=?iso-2022-jp?B?Y2NieQ==?=<ccby@hotmail.com>,<flossie@groucho.cse.psu.edu>,None,None,text/plain,=?iso-2022-jp?B?GyRCRTdBMyRBJGMkcyUyJUMlSBsoQg...,$B#G#W$O$3$&$$$&=w$N;R$H2a$4$7$?$$$C$9!#(Bht...,spam
112,Jul,21,"Wed,",Two,1993,"""Marcus Samuel"" <comments@100-free-sex-picture...",jillian@groucho.cse.psu.edu,None,None,text/plain,fwd: The hottest pick daily news,"Get CTXE First Thing Today, This Is Going To E...",spam
116,[],[],[],[],[],=?iso-2022-jp?B?eXVraWhhbmE=?=<yukihana@yahoo....,<sherri@groucho.cse.psu.edu>,None,None,text/plain,=?iso-2022-jp?B?UmU6?=,$B40(-A4(-L5(-NA(-!u(-;H(-$$(-J|(-Bj(-(B$B(...,spam
123,[],[],[],[],[],=?iso-2022-jp?B?Y2NieQ==?=<ccby@hotmail.com>,<flossie@groucho.cse.psu.edu>,None,None,text/plain,=?iso-2022-jp?B?GyRCRTdBMyRBJGMkcyUyJUMlSBsoQg...,$B#G#W$O$3$&$$$&=w$N;R$H2a$4$7$?$$$C$9!#(Bht...,spam
132,93,Jul,29,Thu,07,stassen@alc-ohio.alc.com (Chris Stassen),christian@geneva.rutgers.edu,,None,None,Re: reading list wanted,Enclosed is Mr. Buxton's proposed creationist ...,ham
145,16:40:34,1993,Aug,3,GMT,huggins@quip.eecs.umich.edu (Jim Huggins),soc-religion-christian@uunet.uu.net,None,None,None,Re: NEEDED: Bible Study Resource(s) on Creatio...,In article <Jul.28.02.36.45.1993.6248@geneva.r...,ham


In [63]:
filepath = 'trec06p%s' % email_nums[9293]
parser = Parser()
with open(filepath, 'r') as ifile:
    email = parser.parse(ifile)
print(email)

From nobody Tue May 30 00:07:44 2017
Received: from media.mit.edu (unknown [80.71.216.37]) by aleve.media.mit.edu
 (8.9.1a/8.9.1/+ALEVE)
 with ESMTP id MAA14280 for <handyboard@media.mit.edu>;
 Fri, 1 Jan 1999 06:06:31 -0500 (EST)
Received: from dogtrot.wilson.ac.jp (exhibit.attune.ac.jp)
 by babysit.tunic.ac.jp (8.11.4/8.11.0av) with ESMTP id g6719Nl6362;
 Fri, 1 Jan 1999 00:03:19 -0700
Message-ID: <mbhxhfzmq@eos.ocn.ne.jp.pt>
Date: Fri, 1 Jan 1999 11:03:19 +0400
From: "Timothy Stephenson" <mbhxhfzmq@eos.ocn.ne.jp>
To: handyboard@media.mit.edu, earline@media.mit.edu, karyn@media.mit.edu,
 sofia@media.mit.edu, eddie@media.mit.edu, janelle@media.mit.edu,
 cherie@media.mit.edu, juan@media.mit.edu, eddie@media.mit.edu
Subject: Boosts Energy
MIME-Version: 1.0
Content-Type: text/plain; charset=ISO-8859-1; format=flowed
Content-Transfer-Encoding: 8bit

Summer is coming, did you look in the mirror lately?

If you`re still overweight you MUST visit us:

http://051.doghealthok.com

WE WILL HELP

In [39]:
Date = email.get('Date')
print(Date)

None


In [36]:
print email

From nobody Mon May 29 23:44:41 2017
Received: from unknown (HELO groucho.cs.psu.edua) ([222.169.109.47]) by
 groucho.cs.psu.edu
 with SMTP id <2590>; Fri, 23 Apr 1993 19:09:18 -0400
To: <winnie@groucho.cs.psu.edu>
From: =?iso-2022-jp?B?d3JxZA==?=<qdrq@yahoo.co.hk>
Subject: =?iso-2022-jp?B?GyRCOSUkKyRsJGs/TSFaJVslQyVIJWklXBsoQlZvbCw0GyRCIVsbKEI=?=
MIME-Version: 1.0
Reply-To: <qadrq@yahoo.co.hk>
Content-Type: text/plain; charset="iso-2022-jp"
Content-Transfer-Encoding: 7bit

$B=w$N;R$+$iCK$N;R$X%*%9%9%a$9$k%J%$%9$JJ8=qH/8+!*(B
http://ivvvvi.com/lab/
$BLd!K(B
fafafasohu@sohu.com





#### Note: $B=w$N;R$+$iCK$N;R$X%*%9%9%a$9$k%J%$%9$JJ8=qH/8+!*(B 

Reason: Chinese

In [44]:
filepath = 'trec06p%s' % email_nums[37804]
parser = Parser()
with open(filepath, 'r') as ifile:
    email = parser.parse(ifile)
    
Date = email.get('Date')
print(email)

From nobody Mon May 29 23:52:48 2017
Received: from unknown (HELO COLUMBIA.EDUa) ([61.47.176.49])
 by longan.cc.columbia.edu (8.13.6/8.13.6) with ESMTP id k4KCxfdv011945
 for <CUSSW-SUBOARD@COLUMBIA.EDU>; Wed, 31 May 2006 16:02:28 -0400 (EDT)
To: <CUSSW-SUBOARD@COLUMBIA.EDU>
From: =?iso-2022-jp?B?GyRCRU9uNE01TGkbKEI=?=<ykokhydmg@orange.livedoor.com>
Subject: =?iso-2022-jp?B?GyRCIVpESTJDJE4kKkNOJGkkOyRHJDkhWxsoQg==?=
MIME-Version: 1.0
Reply-To: <ykokhydmg@orange.livedoor.com>
Content-Type: text/html; charset="iso-2022-jp"
Content-Transfer-Encoding: 7bit

<HTML>
<HEAD>
<META NAME="GENERATOR" Content="Microsoft DHTML Editing Control">
<TITLE></TITLE>
</HEAD>
<BODY>
<P>$BC4Ev$NF#86$G$4$6$$$^$9!#(B<BR>$B?=$7K:$l$?;v$,$"$j$^$7$F:FEY$4O"Mm$5$;$FD:$-$^$7$?!#(B<BR>$B6(NONA6b$O@h$K$G$b$*EO$7=PMh$^$9!#(B<BR>$B$U$_$(MM$h$j3NG'$r<h$i$;$FD:$-B(F|$*?69~$_$5$;$FD:$-$^$9!#(B</P>
<P><A 
href="http://www.wave-pillow.com/box">http://www.wave-pillow.com/box</A><BR>$B$K$F5.J}MM$r$:$C$H$*BT$A$K$J$C$